# Análise H0 e Om0 - flatLCDM

In [ ]:
import os
import h5py 
import json
import numpy as np
import healpy as hp
import pickle
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
from tqdm import tqdm
from scipy.interpolate import interp1d
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import FuncFormatter
from CHIMERA.DataGW import DataLVK
from CHIMERA.Likelihood import LikeLVK
from CHIMERA.astro.mass import pdf_FLAT, dummy_mass
from CHIMERA.astro.rate import phi_PL, dummy_rate
from CHIMERA.cosmo import fLCDM
from CHIMERA.utils import misc, plotting
from astropy.cosmology import FlatLambdaCDM
from scipy.optimize import root
%config InlineBackend.figure_format = "svg"

In [ ]:
dir_LVK       = os.getcwd()
dir_GLADE     = "../../data/GLADE/glade+_GW170817_cutout.hdf5"
dir_GLADE_int = "../../data/GLADE/p_bkg_gauss_smooth_zerr0.001_zres5000_smooth30.pkl"

data_GW_n = "GW170817"                          
data_GW = DataLVK(dir_LVK).load(data_GW_n)      
data_GW["dL"] /= 1000.     

ra4993    = 3.44613079                         
dec4993   = -0.40812585                        
z4993     = 0.0091909380476333                 
errz4993  = 0.0002665372033813657              

In [ ]:
recipe  = {"data_GW_names":     data_GW_n,        
           "data_GW":           data_GW,          
           "data_GW_smooth":    0.3,              
           "data_GAL_dir":      dir_GLADE,        
           "data_GAL_int_dir":  dir_GLADE_int,    
           "data_GAL_weights":  "L_K",            
           "npix_event":        20,               
           "sky_conf":          0.90,             
           "nside_list":        [512,256,128,64,32,16,8],    
           "z_int_sigma":       5,               
           "z_int_res":         300,              
           "z_det_range":       [0.01, 0.5],      
           "z_int_H0_prior":    [30.,220.],       
           "Lcut":              0.02,             
           "band":              "K",
           "completeness":      None,
}

# REPRODUÇÃO DA ANÁLISE (TUTORIAL) DE H_0

In [ ]:
like_dark = LikeLVK(model_cosmo = fLCDM, model_mass  = dummy_mass, model_rate  = dummy_rate, **recipe)

grid_H0 = np.linspace(20.,200.,99)          
like_H0 = np.zeros_like(grid_H0)            

lambda_cosmo = {"Om0": 0.3, "H0": 70.0}    
lambda_mass = {}
lambda_rate = {}

In [ ]:
for i in tqdm(range(len(grid_H0))):    
    lambda_cosmo["H0"] = grid_H0[i]
    like_H0[i]      = like_dark.compute(lambda_cosmo, lambda_mass, lambda_rate, inspect=True)
    
    # Adicionar prints para diagnóstico
    print(f"Iteration {i}")
    print(f"lambda_cosmo: {lambda_cosmo}")
    print(f"like_H0[{i}]: {like_H0[i]}")

# Depois do loop, mostrar todos os resultados
print("Resultados finais:")
print("grid_H0:", grid_H0)
print("like_H0:", like_H0)

In [ ]:
plt.plot(grid_H0, like_H0)
plt.xlabel('$H_0$')
plt.ylabel('Likelihood')
plt.grid(True)
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3.5), dpi=150)

plotting.plot_2Dcoring(like_dark, renorm_all=1, iteration=np.abs(grid_H0 - 70).argmin(), ax=ax)
ax[0].scatter(ra4993, dec4993, edgecolors='k', facecolors='none', s=55, lw=1,  label="NGC 4993")
ax[0].legend()
#plt.savefig('zdist1.jpg')
#plt.close()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5,3.5), dpi=150)

post_H0 = like_H0 / grid_H0**3   

misc.get_confidence_HDI(post_H0, grid_H0, kde=0.2, ax=ax, color="indigo")
plt.xlabel(r"$H_0$ [km/s/Mpc]")
plt.ylabel(r"$p(H_0)$")
plt.legend()
plt.tight_layout()
#plt.savefig('plotH0.jpg')
#plt.close()
plt.show()

# Análise de Om0

In [ ]:
like_dark = LikeLVK(model_cosmo = fLCDM, model_mass  = dummy_mass, model_rate  = dummy_rate, **recipe)          

grid_Om0 = np.linspace(0.01,1.,99)
like_Om0 = np.zeros_like(grid_Om0)

lambda_cosmo = {"Om0": 0.3, "H0": 70.0}    
lambda_mass = {}
lambda_rate = {}

In [ ]:
for i in tqdm(range(len(grid_Om0))):  
    lambda_cosmo["Om0"] = grid_Om0[i]
    like_Om0[i] = like_dark.compute(lambda_cosmo, lambda_mass, lambda_rate, inspect=True)
    
    # Adicionar prints para diagnóstico
    print(f"Iteration {i}")
    print(f"lambda_cosmo: {lambda_cosmo}")
    print(f"like_Om0[{i}]: {like_Om0[i]}")

# Depois do loop, mostrar todos os resultados
print("Resultados finais:")
print("grid_Om0:", grid_Om0)
print("like_Om0:", like_Om0)

In [ ]:
plt.plot(grid_Om0, like_Om0)
plt.xlabel('Densidade de Matéria ($\Omega_m$)')
plt.ylabel('Likelihood')
plt.grid(True)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5,3.5), dpi=150)

post_Om0 = like_Om0 / np.trapz(like_Om0, grid_Om0)     

misc.get_confidence_HDI(post_Om0, grid_Om0, kde=0.2, ax=ax, color="indigo")
plt.xlabel(r"$Om_0$")
plt.ylabel(r"$p(Om_0)$")
#plt.legend()
plt.tight_layout()
#plt.savefig('plotOm.jpg')
#plt.close()
plt.show()